# 1.0 Introduction

## 1.1 Sources, inspirations, and credits

Inspirations:
- [Caltech GPS Basics](http://web.gps.caltech.edu/classes/ge111/Docs/GPSbasics.pdf)
- [2D trilateration GIS stackexchange thread](https://gis.stackexchange.com/questions/40660/trilateration-algorithm-for-n-amount-of-points)
- [Stanford GPS course on youtube](https://www.youtube.com/watch?v=-hnkDQIZ8kY)
    

Credits:
Author(s) - Jon Mayes, [Quantum Logik Consulting](https://quantumlogik.com)

# 2.0 Imports

In [ ]:
#just numpy for this one

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)

In [ ]:
#import qlc color scheme
qlcGreens = ['#001a1a',
            '#003333',
            '#004d4d',
            '#006666',
            '#008080',
            '#009999',
            '#00b3b3',
            '#00cccc',
            '#00e6e6',
            '#00ffff',
            '#1affff',
            '#33ffff',
            '#4dffff',
            '#66ffff',
            '#80ffff',
            '#99ffff',
            '#b3ffff',
            '#ccffff',
            '#e6ffff',
            '#ffffff']

# 3.0 Setup

Let's set up a very simple problem.
- We live in two dimensional space
- we have 4 satelittles, located below (there will be a visual at the end of the section)
- The transmission times are all simultaneous and we have no clock bias or anything fun like that

In [ ]:
sat0 = [0,1]
sat1 = [1,1]
sat2 = [2,1]
sat3 = [2,0]

sats = np.array([sat0,sat1, sat2, sat3])
sats


- The GPS device is located at (1,0)

- The distance from each satellite to the truth is the pseudorange

In [ ]:
truth = np.array([1,0])

pseudorange0 = np.sqrt(2) 
pseudorange1 = 1 
pseudorange2 = np.sqrt(2) 
pseudorange3 = 1 
pseudoranges = np.array([pseudorange0,pseudorange1,pseudorange2, pseudorange3])
pseudoranges

A kludgy graph to visualize the problem

In [ ]:
def viz_problem(sats, prs, truth, estimate):
     
    figure, axes = plt.subplots()
    
    plt.scatter(sats[:,0], sats[:,1]) 
    
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    
    for i in range(len(sats)):
    
        thisSat = sats[i]
        sat_range = plt.Circle( (thisSat[0], thisSat[1] ) , prs[i],
                               color = qlcGreens[i],
                               alpha = .2                          
                              )
        plt.annotate('Sat ' + str(i), # this is the text
                 (thisSat[0], thisSat[1]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center')
              
        axes.add_artist( sat_range )  
       
    plt.plot(truth[0],truth[1],'rX', markersize=16)     
    
    plt.annotate('True Location', # this is the text
                 (truth[0],truth[1]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,0), # distance from text to points (x,y)
                 color='red',
                 ha='right')
    
    
    if estimate is None:
        pass
    else:
        plt.plot(estimate[0],estimate[1],'b*', markersize=12)     
    
        plt.annotate('Estimate', # this is the text
                 (estimate[0],estimate[1]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(5,-1), # distance from text to points (x,y)
                 color='blue',    
                 ha='left')
     
    axes.set_aspect( 1 )
    plt.title( 'Current State of Iteration' )  
    plt.show()
    return 

In [ ]:
viz_problem(sats, pseudoranges, truth, None)


# 4.0 Iterations

## 4.1 Initial guess

The algorithm described in the sources above is an iterative one. That is, we need to create a guess, then the algorithm will point us in the direction in the true location.

As an initial guess, we will take the average of the satellites and label it with a blue star

In [ ]:
sat_average = sats.mean(axis=0)
#init = np.array([1.05,-.14])
#init = estimate

In [ ]:
sat_average

In [ ]:
viz_problem(sats, pseudoranges, truth, sat_average)



With an initial estimate, we know calculate the theoretical ranges, that is, the ranges if our guess was true.

As we will be taking difference, we calculate the residuals, that is the actual measurement versus the theoretical/assumed ranges


In [ ]:
estimate = sat_average

In [ ]:


theorange0 = np.sqrt((estimate[0] - sat0[0])**2 + (estimate[1] - sat0[1])**2) 
theorange1 = np.sqrt((estimate[0] - sat1[0])**2 + (estimate[1] - sat1[1])**2) 
theorange2 = np.sqrt((estimate[0] - sat2[0])**2 + (estimate[1] - sat2[1])**2) 
theorange3 = np.sqrt((estimate[0] - sat3[0])**2 + (estimate[1] - sat3[1])**2) 
theoranges = np.array([theorange0,theorange1,theorange2, theorange3])
theoranges

In [ ]:
residuals = pseudoranges - theoranges
residuals

From the lessons, we know we will need to calculate the design/geometry matrix.

This is G in the Stanford Lecture and A in the Caltech document.

G is the Satellite positions less our estimate, normalized.
We normalize because we only want the direction

We will do this in a few steps to be very clear.


In [ ]:
G_unnorm = sats - estimate
G_unnorm

In [ ]:
norm = np.linalg.norm(G_unnorm)
norm

In [ ]:
G_norm = G_unnorm / norm
G_norm

So, if this were perfectly specified, that is, with our 2 dimension we had only 2 satellites, we could just take the inverse of G.

However, we will find that we will have more satellites than dimensions (which is good since in the real world we won't have such perfect estimates). 

Thus, we create an invertible matrix by front multiplying G by its transpose, which I will call GtG

In [ ]:
G_norm_trans = np.transpose(G_norm)
G_norm_trans

In [ ]:
GtG = G_norm_trans @ G_norm
GtG

In [ ]:
GtGinv = np.linalg.inv(GtG)
GtGinv

In [ ]:
trans_matrix = GtGinv @ G_norm_trans
trans_matrix

In [ ]:
correction = trans_matrix @ residuals
correction

In [ ]:
np.linalg.norm(correction)

In [ ]:
estimate = estimate - correction
estimate

In [ ]:
distance_from_true = np.sqrt((truth[0]-estimate[0])**2 + (truth[1] - estimate[1])**2)
distance_from_true

In [ ]:
distance_from_true_sat_average = np.sqrt((truth[0]-sat_average[0])**2 + (truth[1]-sat_average[1])**2)
distance_from_true_sat_average

So, we got closer, but if we visualize it, we will see that we overshot.

In [ ]:
viz_problem(sats, pseudoranges, truth, estimate)

## 4.2 Iteration 2 using latest estimate

This is an iterative method. Let's run this again with the new estimate

We will include the entire process in one cell for "brevity"

In [ ]:
theorange0 = np.sqrt((estimate[0] - sat0[0])**2 + (estimate[1] - sat0[1])**2) 
theorange1 = np.sqrt((estimate[0] - sat1[0])**2 + (estimate[1] - sat1[1])**2) 
theorange2 = np.sqrt((estimate[0] - sat2[0])**2 + (estimate[1] - sat2[1])**2) 
theorange3 = np.sqrt((estimate[0] - sat3[0])**2 + (estimate[1] - sat3[1])**2) 
theoranges = np.array([theorange0,theorange1,theorange2, theorange3])


residuals = pseudoranges - theoranges

G_unnorm = sats - estimate
norm = np.linalg.norm(G_unnorm)

G_norm_trans = np.transpose(G_norm)
G_norm = G_unnorm / norm

GtG = G_norm_trans @ G_norm
GtGinv = np.linalg.inv(GtG)

trans_matrix = GtGinv @ G_norm_trans
correction = trans_matrix @ residuals

estimate = estimate - correction
estimate

In [ ]:
distance_from_true = np.sqrt((truth[0]-estimate[0])**2 + (truth[1] - estimate[1])**2)
distance_from_true

In [ ]:
viz_problem(sats, pseudoranges, truth, estimate)

In [ ]:
distance_from_true = np.sqrt((truth[0]-estimate[0])**2 + (truth[1] - estimate[1])**2)
distance_from_true

## 4.3 Iteration 3

In [ ]:
theorange0 = np.sqrt((estimate[0] - sat0[0])**2 + (estimate[1] - sat0[1])**2) 
theorange1 = np.sqrt((estimate[0] - sat1[0])**2 + (estimate[1] - sat1[1])**2) 
theorange2 = np.sqrt((estimate[0] - sat2[0])**2 + (estimate[1] - sat2[1])**2) 
theorange3 = np.sqrt((estimate[0] - sat3[0])**2 + (estimate[1] - sat3[1])**2) 
theoranges = np.array([theorange0,theorange1,theorange2, theorange3])


residuals = pseudoranges - theoranges

G_unnorm = sats - estimate
norm = np.linalg.norm(G_unnorm)

G_norm_trans = np.transpose(G_norm)
G_norm = G_unnorm / norm

GtG = G_norm_trans @ G_norm
GtGinv = np.linalg.inv(GtG)

trans_matrix = GtGinv @ G_norm_trans
correction = trans_matrix @ residuals

estimate = estimate - correction
estimate

In [ ]:
viz_problem(sats, pseudoranges, truth, estimate)

In [ ]:
distance_from_true = np.sqrt((truth[0]-estimate[0])**2 + (truth[1] - estimate[1])**2)
distance_from_true

So, what do you notice? 

The algoirthm is pointing us in the right direction, however, we seem to keep overshooting the target. Thus we seem to be destined to flip flop around.

What's the fix?

Think about what we do in other machine learning tasks, particularly when we are using gradient decent. Maybe this algorithm "learns" too fast? 